In [1]:
import numpy as np
import respy as rp
from estimagic.optimization.optimize import minimize

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Parameters for the simulation --> will go to config.py
NUM_AGENTS = 500
NUM_PERIODS = 5

<IPython.core.display.Javascript object>

In the following we will simulate 500 representative Robinson Crusoe agents that live and plan for 5 periods.

In [3]:
params_base, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)
options["n_periods"] = NUM_PERIODS
options["simulation_agents"] = NUM_AGENTS

<IPython.core.display.Javascript object>

**Simulation**
1. Build the simulate function.
2. Enter the parameters into the built simulate function and simulate the agents

In [4]:
simulate = rp.get_simulate_func(params_base, options)
df_base = simulate(params_base)

<IPython.core.display.Javascript object>

The observed data will show that our 500th agent will alternate between hammock and fishing. In the first two periods he will idle around in the hammock. In period 2 he starts to go fishing in order to make a living. However, in the last period our Robinson No. 500 decides to go into retirement in the hammock.

In [5]:
# df_base.tail(10)

<IPython.core.display.Javascript object>

We "select" our moments that we will use for the MSM procedure
- Choice frequencies
- Wage distribution

In [6]:
def calc_choice_frequencies(df):
    """Calculation of choice frequencies"""
    return df.groupby("Period").Choice.value_counts(normalize=True).unstack()

<IPython.core.display.Javascript object>

In [7]:
def calc_wage_distribution(df):
    """Calculation of wage distribution"""
    return df.groupby(["Period"])["Wage"].describe()[["mean", "std"]]

<IPython.core.display.Javascript object>

In [8]:
def fill_nans_zero(df):
    return df.fillna(0)

<IPython.core.display.Javascript object>

Assemble the desired calculation wrapper for moments. Currently supported methods are
- `calc_choice_frequencies`
- `calc_wage_distribution`

In [9]:
calc_moments = {
    "Choice Frequencies": calc_choice_frequencies,
    "Wage Distribution": calc_wage_distribution,
}

<IPython.core.display.Javascript object>

We need to define how to deal with missings: We will just replace them with 0-values.

In [10]:
def replace_nans(df):
    return df.fillna(0)

<IPython.core.display.Javascript object>

In [11]:
moments_obs = {
    "Choice Frequencies": replace_nans(calc_moments["Choice Frequencies"](df_base)),
    "Wage Distribution": replace_nans(calc_moments["Wage Distribution"](df_base)),
}

<IPython.core.display.Javascript object>

In [12]:
# print('Choice Frequencies')
# print(moments_obs["Choice Frequencies"])
# print('\n Wage Distribution')
# print(moments_obs["Wage Distribution"])

<IPython.core.display.Javascript object>

In [13]:
def get_weighting_matrix(data, calc_moments, num_boots, num_agents_msm):
    """ Compute weighting matrix for estimation with MSM."""
    # Seed for reproducibility.
    np.random.seed(123)

    index_base = data.index.get_level_values("Identifier").unique()

    # Create bootstrapped moments.
    moments_sample = list()
    for _ in range(num_boots):
        ids_boot = np.random.choice(index_base, num_agents_msm, replace=False)
        moments_boot = [
            calc_moments[key](data.loc[ids_boot, :]) for key in calc_moments.keys()
        ]

        moments_boot = rp.get_flat_moments(moments_boot)

        moments_sample.append(moments_boot)

    # Compute variance for each moment and construct diagonal weighting matrix.
    moments_var = np.array(moments_sample).var(axis=0)
    weighting_matrix = np.diag(moments_var ** (-1))

    return np.nan_to_num(weighting_matrix)

<IPython.core.display.Javascript object>

In [14]:
weighting_matrix = rp.get_diag_weighting_matrix(moments_obs)

<IPython.core.display.Javascript object>

In [15]:
msm = rp.get_msm_func(
    params=params_base,
    options=options,
    calc_moments=calc_moments,
    replace_nans=fill_nans_zero,
    empirical_moments=moments_obs,
    weighting_matrix=weighting_matrix,
    return_scalar=True,
)

<IPython.core.display.Javascript object>

In [16]:
criterion_msm_base = rp.get_msm_func(
    params_base, options, calc_moments, replace_nans, moments_obs, weighting_matrix
)

<IPython.core.display.Javascript object>

In [17]:
fval = criterion_msm_base(params_base)
fval

0.0

<IPython.core.display.Javascript object>

## Alternate discounting of agent

In [18]:
params_delta = params_base.copy()
params_delta.loc["delta", "value"] = 0.8

<IPython.core.display.Javascript object>

In [19]:
simulate_delta = rp.get_simulate_func(params_delta, options)
df_sim_delta = simulate_delta(params_delta)

<IPython.core.display.Javascript object>

In [20]:
moments_sim_delta = {
    "Choice Frequencies": replace_nans(
        calc_moments["Choice Frequencies"](df_sim_delta)
    ),
    "Wage Distribution": replace_nans(calc_moments["Wage Distribution"](df_sim_delta)),
}

<IPython.core.display.Javascript object>

In [21]:
criterion_msm_delta = rp.get_msm_func(
    params_delta, options, calc_moments, replace_nans, moments_obs, weighting_matrix
)

<IPython.core.display.Javascript object>

In [22]:
fval = criterion_msm_delta(params_delta)
fval

0.3065964459758088

<IPython.core.display.Javascript object>

## Inclusion of ambiguity

Open question: When we add ambiguity, do we need to calculate the moments, weighting matrix etc. anew? Conjecture: Yes, we should.

In [23]:
params_eta = params_base.copy()
params_eta.loc[("eta", "eta"), :] = 0.1  # eta_values["baseline"]
# params_eta

<IPython.core.display.Javascript object>

In [24]:
simulate_eta = rp.get_simulate_func(params_eta, options)
df_sim_eta = simulate_delta(params_eta)

<IPython.core.display.Javascript object>

As we have simulated new agents with including ambiguity, we need to calculate the simulated moments based on the resulting "simulation data frame" `df_sim_eta`.

In [25]:
moments_sim_eta = {
    "Choice Frequencies": replace_nans(calc_moments["Choice Frequencies"](df_sim_eta)),
    "Wage Distribution": replace_nans(calc_moments["Wage Distribution"](df_sim_eta)),
}

<IPython.core.display.Javascript object>

Build the MSM criterion function, including the $\eta$ parameters.

In [26]:
criterion_msm_eta = rp.get_msm_func(
    params_eta, options, calc_moments, replace_nans, moments_obs, weighting_matrix
)

<IPython.core.display.Javascript object>

In [27]:
rslt = minimize(
    criterion=criterion_msm_eta, params=params_eta, algorithm="nlopt_bobyqa"
)

<IPython.core.display.Javascript object>

In [28]:
rslt[0]

{'fun': 0.0}

<IPython.core.display.Javascript object>

ESTIMATION FIXING CONSTRAINTS

In [29]:
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"},
    {"loc": "eta", "type": "fixed"},
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type": "fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

<IPython.core.display.Javascript object>

In [30]:
constr_eta = constr_base.copy()
constr_eta.remove({"loc": "eta", "type": "fixed"},)

<IPython.core.display.Javascript object>

In [31]:
rslt = minimize(
    criterion=criterion_msm_eta,
    params=params_eta,
    algorithm="nlopt_bobyqa",
    constraints=constr_eta,
)
rslt[1]

value  lower  upper  \
category                name                                           
delta                   delta                     0.95   -inf    inf   
wage_fishing            exp_fishing               0.10   -inf    inf   
nonpec_fishing          constant                 -1.00   -inf    inf   
nonpec_hammock          constant                  2.50   -inf    inf   
                        not_fishing_last_period  -1.00   -inf    inf   
shocks_sdcorr           sd_fishing                1.00   -inf    inf   
                        sd_hammock                1.00   -inf    inf   
                        corr_hammock_fishing     -0.20   -inf    inf   
lagged_choice_1_hammock constant                  1.00   -inf    inf   
inadmissibility_penalty inadmissibility_penalty -20.00   -inf    inf   
eta                     eta                       0.00   -inf    inf   

                                                          group  \
category                name                                      
delta                   delta                    All Parameters   
wage_fishing            exp_fishing              All Parameters   
nonpec_fishing          constant                 All Parameters   
nonpec_hammock          constant                 All Parameters   
                        not_fishing_last_period  All Parameters   
shocks_sdcorr           sd_fishing               All Parameters   
                        sd_hammock               All Parameters   
                        corr_hammock_fishing     All Parameters   
lagged_choice_1_hammock constant                 All Parameters   
inadmissibility_penalty inadmissibility_penalty  All Parameters   
eta                     eta                      All Parameters   

                                                                                            name  \
category                name                                                                       
delta                   delta                                                        delta_delta   
wage_fishing            exp_fishing                                     wage_fishing_exp_fishing   
nonpec_fishing          constant                                         nonpec_fishing_constant   
nonpec_hammock          constant                                         nonpec_hammock_constant   
                        not_fishing_last_period           nonpec_hammock_not_fishing_last_period   
shocks_sdcorr           sd_fishing                                      shocks_sdcorr_sd_fishing   
                        sd_hammock                                      shocks_sdcorr_sd_hammock   
                        corr_hammock_fishing                  shocks_sdcorr_corr_hammock_fishing   
lagged_choice_1_hammock constant                                lagged_choice_1_hammock_constant   
inadmissibility_penalty inadmissibility_penalty  inadmissibility_penalty_inadmissibility_penalty   
eta                     eta                                                              eta_eta   

                                                 _fixed_value  \
category                name                                    
delta                   delta                            0.95   
wage_fishing            exp_fishing                      0.10   
nonpec_fishing          constant                        -1.00   
nonpec_hammock          constant                         2.50   
                        not_fishing_last_period         -1.00   
shocks_sdcorr           sd_fishing                       1.00   
                        sd_hammock                       1.00   
                        corr_hammock_fishing            -0.20   
lagged_choice_1_hammock constant                          NaN   
inadmissibility_penalty inadmissibility_penalty           NaN   
eta                     eta                               NaN   

                                                 _is_fixed_to_value  \
category              

<IPython.core.display.Javascript object>

In [32]:
# Check the parameter combinations
eta_perturbed = {
    "true": [],
    "estimated": [],
}

<IPython.core.display.Javascript object>

In [ ]:
for eta in np.linspace(0, 0.25, 11):

    # We want a change in the starting values (not in the simulated data)
    params_eta.loc[("eta", "eta"), :] = eta

    # params["group"] = params.index.get_level_values('category')
    # need above only if we want dashboard output

    # Get the criterion function with simulated data, but different param values
    criterion_msm_eta = rp.get_msm_func(
        params_eta, options, calc_moments, replace_nans, moments_obs, weighting_matrix
    )

    rslt, params_rslt = minimize(
        criterion=criterion_msm_eta, params=params_eta, algorithm="nlopt_bobyqa"
    )

    eta_perturbed["true"].append(eta)
    eta_perturbed["estimated"].append(params_rslt.loc["eta", "value"][0])